# Google Drive Datei- und Verzeichnis-Suchtool

Ein praktisches Projekt zur Anwendung von:
- **Rekursion**: Durchqueren des Verzeichnisbaums
- **Decorator**: Laufzeitmessung der Suchfunktionen
- **Closures**: Erweiterte Suchfunktionalität

## Funktionen
- Rekursive Suche nach Dateien nach Erweiterung in Google Drive
- Suche nach Dateinamen (exakt oder Teilstring) in Google Drive
- Laufzeitmessung mit Decorator
- Flexible Suchoptionen

**Hinweis**: Dieses Notebook ist speziell für die Suche in Google Drive optimiert. Google Drive wird automatisch gemountet.


In [ ]:
import os
import time
from pathlib import Path
from functools import wraps
from typing import List, Callable, Optional

# Prüfe ob wir auf Google Colab laufen
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colab erkannt!")
except ImportError:
    IN_COLAB = False
    print("⚠️  Warnung: Dieses Notebook ist für Google Colab optimiert!")
    print("   Bitte führen Sie es auf Google Colab aus.")


## Google Drive Verbindung

**Wichtig**: Dieses Notebook verbindet sich automatisch mit Google Drive und sucht nur dort nach Dateien.

- Google Drive wird automatisch gemountet
- Der Standardpfad ist `/content/drive/MyDrive`
- Alle Suchfunktionen durchsuchen ausschließlich Google Drive
- Sie müssen die Authentifizierung einmalig durchführen


In [ ]:
# Google Drive mounten
from google.colab import drive

# Mounte Google Drive
# Sie werden aufgefordert, sich zu authentifizieren und den Zugriff zu erlauben
drive.mount('/content/drive')

# Definiere den Standardpfad für Google Drive
DRIVE_PATH = '/content/drive/MyDrive'

# Prüfe ob Drive erfolgreich gemountet wurde
if os.path.exists(DRIVE_PATH):
    print(f"✅ Google Drive erfolgreich gemountet!")
    print(f"📁 Drive-Pfad: {DRIVE_PATH}")
    print(f"📊 Anzahl der Elemente im Hauptverzeichnis: {len(list(Path(DRIVE_PATH).iterdir()))}")
else:
    print("⚠️  Warnung: Google Drive konnte nicht gemountet werden!")


## Decorator für Laufzeitmessung

Ein Decorator, der die Ausführungszeit einer Funktion misst und anzeigt.


In [ ]:
def laufzeit_messung(func: Callable) -> Callable:
    """
    Decorator zur Messung der Ausführungszeit einer Funktion.
    
    Args:
        func: Die zu messende Funktion
        
    Returns:
        Wrapped Funktion mit Laufzeitmessung
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        laufzeit = end_time - start_time
        print(f"⏱️  Funktion '{func.__name__}' benötigte {laufzeit:.4f} Sekunden")
        return result
    return wrapper


## Rekursive Suchfunktionen

Die Hauptfunktionen zur rekursiven Durchsuchung des Verzeichnisbaums.


In [ ]:
@laufzeit_messung
def suche_dateien_nach_erweiterung(verzeichnis: str, erweiterung: str) -> List[str]:
    """
    Sucht rekursiv nach allen Dateien mit einer bestimmten Erweiterung.
    
    Args:
        verzeichnis: Der Startpfad für die Suche
        erweiterung: Die Dateierweiterung (z.B. '.txt', '.py')
        
    Returns:
        Liste aller gefundenen Dateipfade
    """
    gefundene_dateien = []
    
    # Normalisiere die Erweiterung (füge Punkt hinzu falls nicht vorhanden)
    if not erweiterung.startswith('.'):
        erweiterung = '.' + erweiterung
    
    def _rekursive_suche(pfad: Path):
        """Innere rekursive Funktion (Closure)"""
        try:
            # Durchlaufe alle Einträge im aktuellen Verzeichnis
            for eintrag in pfad.iterdir():
                if eintrag.is_file():
                    # Prüfe ob die Datei die gewünschte Erweiterung hat
                    if eintrag.suffix.lower() == erweiterung.lower():
                        gefundene_dateien.append(str(eintrag))
                elif eintrag.is_dir():
                    # Rekursiver Aufruf für Unterverzeichnisse
                    _rekursive_suche(eintrag)
        except PermissionError:
            # Überspringe Verzeichnisse ohne Zugriffsrechte
            pass
    
    start_pfad = Path(verzeichnis)
    if not start_pfad.exists():
        print(f"⚠️  Warnung: Verzeichnis '{verzeichnis}' existiert nicht!")
        return []
    
    _rekursive_suche(start_pfad)
    return gefundene_dateien


In [ ]:
@laufzeit_messung
def suche_dateien_nach_name(verzeichnis: str, name_pattern: str, exakt: bool = False) -> List[str]:
    """
    Sucht rekursiv nach Dateien, deren Name einem Muster entspricht.
    
    Args:
        verzeichnis: Der Startpfad für die Suche
        name_pattern: Der zu suchende Dateiname oder Teilstring
        exakt: Wenn True, muss der Name exakt übereinstimmen
        
    Returns:
        Liste aller gefundenen Dateipfade
    """
    gefundene_dateien = []
    
    def _rekursive_suche(pfad: Path):
        """Innere rekursive Funktion (Closure)"""
        try:
            for eintrag in pfad.iterdir():
                if eintrag.is_file():
                    dateiname = eintrag.name
                    if exakt:
                        if dateiname == name_pattern:
                            gefundene_dateien.append(str(eintrag))
                    else:
                        if name_pattern.lower() in dateiname.lower():
                            gefundene_dateien.append(str(eintrag))
                elif eintrag.is_dir():
                    _rekursive_suche(eintrag)
        except PermissionError:
            pass
    
    start_pfad = Path(verzeichnis)
    if not start_pfad.exists():
        print(f"⚠️  Warnung: Verzeichnis '{verzeichnis}' existiert nicht!")
        return []
    
    _rekursive_suche(start_pfad)
    return gefundene_dateien


In [ ]:
@laufzeit_messung
def suche_dateien_erweitert(verzeichnis: str, 
                            erweiterung: Optional[str] = None,
                            name_enthält: Optional[str] = None,
                            min_größe: Optional[int] = None,
                            max_größe: Optional[int] = None) -> List[dict]:
    """
    Erweiterte Suchfunktion mit mehreren Filtern.
    
    Args:
        verzeichnis: Der Startpfad für die Suche
        erweiterung: Optional: Dateierweiterung (z.B. '.txt')
        name_enthält: Optional: Teilstring im Dateinamen
        min_größe: Optional: Minimale Dateigröße in Bytes
        max_größe: Optional: Maximale Dateigröße in Bytes
        
    Returns:
        Liste von Dictionaries mit Dateiinformationen
    """
    gefundene_dateien = []
    
    def _rekursive_suche(pfad: Path):
        """Innere rekursive Funktion (Closure)"""
        try:
            for eintrag in pfad.iterdir():
                if eintrag.is_file():
                    # Prüfe alle Filter
                    passt = True
                    
                    if erweiterung:
                        norm_erw = erweiterung if erweiterung.startswith('.') else '.' + erweiterung
                        if eintrag.suffix.lower() != norm_erw.lower():
                            passt = False
                    
                    if passt and name_enthält:
                        if name_enthält.lower() not in eintrag.name.lower():
                            passt = False
                    
                    if passt and (min_größe is not None or max_größe is not None):
                        datei_größe = eintrag.stat().st_size
                        if min_größe is not None and datei_größe < min_größe:
                            passt = False
                        if max_größe is not None and datei_größe > max_größe:
                            passt = False
                    
                    if passt:
                        gefundene_dateien.append({
                            'pfad': str(eintrag),
                            'name': eintrag.name,
                            'größe': eintrag.stat().st_size,
                            'erweiterung': eintrag.suffix
                        })
                        
                elif eintrag.is_dir():
                    _rekursive_suche(eintrag)
        except PermissionError:
            pass
    
    start_pfad = Path(verzeichnis)
    if not start_pfad.exists():
        print(f"⚠️  Warnung: Verzeichnis '{verzeichnis}' existiert nicht!")
        return []
    
    _rekursive_suche(start_pfad)
    return gefundene_dateien


## Beispiele für die Suche in Google Drive

### Beispiel 1: Suche nach .txt-Dateien


In [ ]:
# Suche nach allen .txt-Dateien in Google Drive
drive_verzeichnis = DRIVE_PATH  # Google Drive Hauptverzeichnis

txt_dateien = suche_dateien_nach_erweiterung(drive_verzeichnis, '.txt')
print(f"\n📄 Gefundene .txt-Dateien in Google Drive: {len(txt_dateien)}")
for datei in txt_dateien[:10]:  # Zeige maximal 10 Dateien
    print(f"  - {datei}")
if len(txt_dateien) > 10:
    print(f"  ... und {len(txt_dateien) - 10} weitere")


### Beispiel 2: Suche nach Python-Dateien (.py)


In [ ]:
# Suche nach allen Python-Dateien in Google Drive
py_dateien = suche_dateien_nach_erweiterung(drive_verzeichnis, 'py')
print(f"\n🐍 Gefundene Python-Dateien in Google Drive: {len(py_dateien)}")
for datei in py_dateien[:10]:
    print(f"  - {datei}")


### Beispiel 3: Suche nach Dateinamen


In [ ]:
# Suche nach Dateien in Google Drive, die "test" im Namen enthalten
test_dateien = suche_dateien_nach_name(drive_verzeichnis, 'test', exakt=False)
print(f"\n🔍 Gefundene Dateien mit 'test' im Namen in Google Drive: {len(test_dateien)}")
for datei in test_dateien[:10]:
    print(f"  - {datei}")


### Beispiel 4: Erweiterte Suche mit mehreren Filtern


In [ ]:
# Erweiterte Suche in Google Drive: .txt-Dateien, die "readme" im Namen enthalten
# und zwischen 100 Bytes und 1 MB groß sind
ergebnisse = suche_dateien_erweitert(
    drive_verzeichnis,
    erweiterung='.txt',
    name_enthält='readme',
    min_größe=100,
    max_größe=1024*1024  # 1 MB
)

print(f"\n📊 Gefundene Dateien in Google Drive: {len(ergebnisse)}")
for datei_info in ergebnisse[:10]:
    größe_kb = datei_info['größe'] / 1024
    print(f"  - {datei_info['name']} ({größe_kb:.2f} KB)")
    print(f"    Pfad: {datei_info['pfad']}")


### Beispiel 5: Suche in spezifischen Google Drive Verzeichnissen

Durchsuchen Sie spezifische Unterverzeichnisse in Google Drive:


In [ ]:
# Suche in Google Drive nach Notebook-Dateien
drive_notebooks = suche_dateien_nach_erweiterung(DRIVE_PATH, '.ipynb')
print(f"\n📓 Gefundene Notebook-Dateien in Google Drive: {len(drive_notebooks)}")
for datei in drive_notebooks[:10]:
    print(f"  - {datei}")

# Beispiel: Suche in einem spezifischen Unterverzeichnis in Google Drive
# Ersetzen Sie 'Ihr_Ordner' mit dem tatsächlichen Ordnernamen
# spezifischer_ordner = f"{DRIVE_PATH}/Ihr_Ordner"
# csv_dateien = suche_dateien_nach_erweiterung(spezifischer_ordner, '.csv')


## Zusammenfassung

Dieses Tool demonstriert:

✅ **Rekursion**: Die Funktionen `_rekursive_suche()` durchlaufen rekursiv den gesamten Verzeichnisbaum in Google Drive

✅ **Closures**: Die inneren Funktionen `_rekursive_suche()` sind Closures, die auf die äußeren Variablen (`gefundene_dateien`, Parameter) zugreifen

✅ **Decorator**: Der `@laufzeit_messung` Decorator misst automatisch die Ausführungszeit jeder Suchfunktion

✅ **Praktische Anwendung**: Flexibles Dateisuchsystem für Google Drive mit verschiedenen Filtermöglichkeiten


## Beispiel: Suche in einem spezifischen Google Drive Ordner

Sie können auch in einem bestimmten Unterordner Ihres Google Drives suchen:


In [ ]:
# Beispiel: Suche in einem spezifischen Ordner in Google Drive
# Passen Sie den Ordnernamen an Ihre Google Drive Struktur an

# Beispiel 1: Suche in einem bestimmten Ordner
# ordner_name = "Meine_Dokumente"  # Ersetzen Sie dies mit Ihrem Ordnernamen
# spezifischer_pfad = f"{DRIVE_PATH}/{ordner_name}"
# 
# if os.path.exists(spezifischer_pfad):
#     pdf_dateien = suche_dateien_nach_erweiterung(spezifischer_pfad, '.pdf')
#     print(f"\n📄 Gefundene PDF-Dateien in '{ordner_name}': {len(pdf_dateien)}")
#     for datei in pdf_dateien[:10]:
#         print(f"  - {datei}")
# else:
#     print(f"⚠️  Ordner '{ordner_name}' existiert nicht in Google Drive")

# Beispiel 2: Zeige verfügbare Ordner im Hauptverzeichnis
print("📁 Verfügbare Ordner in Google Drive:")
try:
    ordner = [item for item in Path(DRIVE_PATH).iterdir() if item.is_dir()]
    for i, o in enumerate(ordner[:20], 1):  # Zeige maximal 20 Ordner
        print(f"  {i}. {o.name}")
    if len(ordner) > 20:
        print(f"  ... und {len(ordner) - 20} weitere Ordner")
except Exception as e:
    print(f"⚠️  Fehler beim Lesen der Ordner: {e}")


In [ ]:
# Beispiel: Erweiterte Suche in einem spezifischen Google Drive Ordner
# Passen Sie den Ordnernamen und die Suchkriterien an

# Beispiel-Suche nach Bilddateien in einem bestimmten Ordner
# ordner_name = "Bilder"  # Ersetzen Sie dies mit Ihrem Ordnernamen
# bilder_pfad = f"{DRIVE_PATH}/{ordner_name}"
# 
# if os.path.exists(bilder_pfad):
#     bilder = suche_dateien_erweitert(
#         bilder_pfad,
#         erweiterung='.jpg',  # oder '.png', '.gif', etc.
#         min_größe=10000,  # Mindestens 10 KB
#         max_größe=10*1024*1024  # Maximal 10 MB
#     )
#     print(f"\n🖼️  Gefundene Bilddateien in '{ordner_name}': {len(bilder)}")
#     for bild_info in bilder[:10]:
#         größe_mb = bild_info['größe'] / (1024 * 1024)
#         print(f"  ✓ {bild_info['name']} - {größe_mb:.2f} MB")
# else:
#     print(f"⚠️  Ordner '{ordner_name}' existiert nicht in Google Drive")

print("\n💡 Tipp: Kommentieren Sie die obigen Beispiele aus und passen Sie die Ordnernamen an!")
